# 1.Multigpu Distributed Training-ScriptMode
---

본 모듈에서는 Amzaon SageMaker API을 효과적으로 이용하기 위해 multigpu-distributed 학습을 위한 PyTorch 프레임워크 자체 구현만으로 모델 훈련을 수행해 봅니다.

## 1. Sagemaker notebook 설명
<p>Sagemaker notebook은 완전 관리형 서비스로 컨테이너 기반으로 구성되어 있습니다. 사용자가 직접 컨테이너를 볼 수 없지만, 내부적으로는 아래와 같은 원리로 동작합니다. </p>
<p><img src="./imgs/fig00.png" width="800", height="80"></p>

- **S3 (Simple Storage Serivce)** : Object Storage로서 학습할 데이터 파일과 학습 결과인 model, checkpoint, tensorboard를 위한 event 파일, 로그 정보 등을 저장하는데 사용합니다.
- **SageMaker Notebook** : 학습을 위한 스크립트 작성과 디버깅, 그리고 실제 학습을 수행하기 위한 Python을 개발하기 위한 환경을 제공합니다.
- **Amazon Elastic Container Registry(ECR)** :  Docker 컨테이너 이미지를 손쉽게 저장, 관리 및 배포할 수 있게 해주는 완전관리형 Docker 컨테이너 레지스트리입니다. Sagemaker는 기본적인 컨테이너를 제공하기 때문에 별도 ECR에 컨테이너 이미지를 등록할 필요는 없습니다. 하지만, 별도의 학습 및 배포 환경이 필요한 경우 custom 컨테이너 이미지를 만들어서 ECR에 등록한 후 이 환경을 활용할 수 있습니다.

<p>학습과 추론을 하는 hosting 서비스는 각각 다른 컨테이너 환경에서 수행할 수 있으며, 쉽게 다량으로 컨테이너 환경을 확장할 수 있으므로 다량의 학습과 hosting이 동시에 가능합니다.   
</p>

In [ ]:
import subprocess

result = subprocess.run(['df','-h'], stdout=subprocess.PIPE)
print(result.stdout.decode('utf-8'))

In [ ]:
install_needed = True  # should only be True once
install_needed = False

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U split-folders tqdm albumentations crc32c wget
    # !{sys.executable} -m pip install 'sagemaker[local]' --upgrade
    !{sys.executable} -m pip install -U bokeh smdebug sagemaker-experiments
    !{sys.executable} -m pip install sagemaker
    IPython.Application.instance().kernel.do_shutdown(True)

## 2. 환경 설정

<p>Sagemaker 학습에 필요한 기본적인 package를 import 합니다. </p>
<p>boto3는 HTTP API 호출을 숨기는 편한 추상화 모델을 가지고 있고, Amazon EC2 인스턴스 및 S3 버켓과 같은 AWS 리소스와 동작하는 파이선 클래스를 제공합니다. </p>
<p>sagemaker python sdk는 Amazon SageMaker에서 기계 학습 모델을 교육 및 배포하기 위한 오픈 소스 라이브러리입니다.</p>

In [ ]:
import joblib
import matplotlib.pyplot as plt
import sagemaker
# import splitfolders

import datetime
import glob
import os
import time
import warnings

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

# import wget
# import tarfile
import shutil

import boto3
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision

# from tqdm import tqdm
from time import strftime
from PIL import Image
from torch.utils.data import Dataset
from torchvision import datasets, transforms

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

from sagemaker.debugger import (Rule,
                                rule_configs,
                                ProfilerConfig, 
                                FrameworkProfile, 
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig)

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
sagemaker.__version__

In [ ]:
sess = boto3.Session()
sagemaker_session = sagemaker.Session()
sm = sess.client('sagemaker')
role = get_execution_role()

In [ ]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name,
                                          tags=[
                                              {
                                                  'Key': 'multigpu',
                                                  'Value': 'yes'
                                              },
                                              {
                                                  'Key': 'multinode',
                                                  'Value': 'yes'
                                              },
                                          ])

In [ ]:
def create_trial(experiment_name, set_param, i_type, i_cnt, spot):
    create_date = strftime("%m%d-%H%M")
    
    spot = 's' if spot else 'd'
    i_tag = 'test'
    if i_type == 'ml.p3.16xlarge':
        i_tag = 'p3'
    elif i_type == 'ml.p3dn.24xlarge':
        i_tag = 'p3dn'
    elif i_type == 'ml.p4d.24xlarge':
        i_tag = 'p4d'    
        

    bs = set_param['BATCH_SIZE']
    trial = "-".join([i_tag,str(i_cnt),str(bs),spot])
       
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{trial}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

In [ ]:
bucket = 'bucket-exp-dalle-210410'
code_location = f's3://{bucket}/sm_codes'
output_path = f's3://{bucket}/poc_dalle/output/' 

In [ ]:
metric_definitions=[
     {'Name': 'train:lr', 'Regex': 'lr - (.*?),'},
     {'Name': 'train:Loss', 'Regex': 'loss -(.*?),'},
]

In [ ]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules=[ 
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overfit()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [ ]:
hyperparameters = {
        'EPOCHS' : 2,
        'BATCH_SIZE' : 8,
        'LEARNING_RATE' : 1e-3,
        'LR_DECAY_RATE' : 0.98,
        'NUM_TOKENS' : 8192,
        'NUM_LAYERS' : 2,
        'NUM_RESNET_BLOCKS' : 2,
        'SMOOTH_L1_LOSS' : False,
        'EMB_DIM' : 512,
        'HID_DIM' : 256,
        'KL_LOSS_WEIGHT' : 0,
        'STARTING_TEMP' : 1.,
        'TEMP_MIN' : 0.5,
        'ANNEAL_RATE' : 1e-6,
        'NUM_IMAGES_SAVE' : 4,
#         'model_parallel': True  ## False : DeepSpeeds
    }

experiment_name = 'dalle-poc-exp1'
train_instance_type = 'ml.p3.16xlarge'  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge'
train_instance_count = 1
do_spot_training = False
max_wait = None
max_run = 2*60*60


In [ ]:
image_uri = None
distribution = None
train_job_name = 'sagemaker'

if hyperparameters.get('data_parallel'):
    train_job_name = 'sdp-dist'
    
    distribution = {"smdistributed": {
                        "dataparallel": {
                                "enabled": True
                        }
                   }
                 }
elif hyperparameters.get('model_parallel'):
    train_job_name = 'smp-dist'
    
    mpioptions = "-verbose -x orte_base_help_aggregate=0 "
    distribution = {"smdistributed": {
                      "modelparallel": {
                          "enabled":True,
                          "parameters": {
                              "microbatches": 8,
                              "placement_strategy": "spread",
                              "pipeline": "interleaved",
                              "optimize": "speed",
                              "partitions": 1,
                              "ddp": True,
                          }
                      }
                  },
                  "mpi": {
                        "enabled": True,
                        "processes_per_host": 1, # Pick your processes_per_host
                        "custom_mpi_options": mpioptions
                  },
              }
    
    profiler_config = None  # smdebug doesnt support detailed profiling with sagemaker model parallel
elif hyperparameters.get('apex'):
    train_job_name = 'apex-dist'
    
else:
    train_instance_type = 'ml.p3.2xlarge'

if do_spot_training:
    max_wait = max_run

print("train_job_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {} \nimage_uri : {} \ndistribution : {}".format(train_job_name, train_instance_type, train_instance_count, image_uri, distribution))    

In [ ]:
%%time

# all input configurations, parameters, and metrics specified in estimator 
# definition are automatically tracked
estimator = PyTorch(
    entry_point='train_vae.py',
    source_dir='source_code',
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version='1.6.0',
    py_version='py36',
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=1024,
    code_location = code_location,
    output_path=output_path,
    hyperparameters=hyperparameters,
    distribution=distribution,
#     disable_profiler=True,
    metric_definitions=metric_definitions,
    rules=rules,
    max_run=max_run,
    use_spot_instances=do_spot_training,  # spot instance 활용
    max_wait=max_wait,
)

In [ ]:
bucket_name = 'dataset-cyj-coco-210410'
s3_data_path = f's3://{bucket_name}/dataset'
s3_data_path

In [ ]:
input_data = sagemaker.inputs.TrainingInput(
        s3_data=s3_data_path,
        distribution='ShardedByS3Key',
        s3_data_type='S3Prefix',
        input_mode='File',
        shuffle_config=sagemaker.inputs.ShuffleConfig(123)
        )

In [ ]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name, hyperparameters, train_instance_type, train_instance_count, do_spot_training)

# Now associate the estimator with the Experiment and Trial
estimator.fit(
    inputs={'training': input_data}, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=False,
)

In [ ]:
job_name=estimator.latest_training_job.name

<p><strong>Aynchronous</strong>로 진행된 Training job은 아래와 같은 방법으로 진행상황을 실시간으로 확인할 수 있습니다.</p>

In [ ]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

In [ ]:
!aws s3 ls {rule_output_path}/ProfilerReport/profiler-output/

In [ ]:
!aws s3 cp {rule_output_path}/ProfilerReport/profiler-output/ {output_dir}/ProfilerReport/ --recursive

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>ProfilerReport : <a href="{}profiler-report.html">Profiler Report</a></b>'.format(output_dir+"/ProfilerReport/")))


In [ ]:
%store hyperparameters model_dir output_dir artifacts_dir

<p></p>
<p>Amazon SageMaker에서 모든 학습을 완료하였습니다. </p>